This project will develop and explore a simple simulation of a health service provider, such as a clinic.

The following assumptions will be made:

Patients arrive at random intervals according to a Poisson process, the time between arrivals having an exponential distribution.

Patients will be randomly assigned a category of health condition. 

Patients may have to wait for a doctor to be available to assess them.

When a doctor is available the patients are assessed and assigned to a specialist doctor/consultant according to the previously assigned condition. The number of doctors performing the initial assessment is a variable to be explored with the simulation.

Patients may have to wait for a specialist to be available.

When a specialist is available the patients are treated. The number of specialists for each category is a variable to be explored by the simulation.

It is assumed that no patients need emergency care so there is no need to prioritize them.

The clinic is open for 7 hours a day. Staff breaks will be ingnored.

In [1]:
# import the required libraries
import simpy
import random
import pandas as pd
import statistics

The simulation will explore three performance indicators; total number of patients treated, mean patient waiting time and idle time for medical staff.

Each simulation will be repeated N times and the mean result calculated for each performance indicator.

In [2]:
# The outcomes for each simulation will be saved in a list
total_patients = []
mean_waiting = []
idle_time = []

# The simulation will be repeated N times
N = 2

The simulation will include:

4 Resources; the doctors conducting the initial assessment and three categories of specialists.

In [3]:
# The parameters are:

# The mean time between patient arrivals (the unit of time used is minutes)
arrival_mean = 5
# The distribution of patient health conditions (probability, total sums to 1)
cond_1_prob = 0.33
cond_2_prob = 0.33
cond_3_prob = 0.34
# The mean time for a doctor to complete the initial assessment of each patient and corresponding lower and upper limits
# Allowing for a uniform distribution and a range of 4 minutes. The minimum mean value is 3 minutes.
assess_mean = 5
assess_lower = assess_mean - 2
assess_upper = assess_mean + 2
# The mean time and standard deviation for a specialist to treat each health condition.
# Assuming a normal distribution and discounting the minimal possibility of a negative time.
special_1_mean = 20
special_1_sd = 5

special_2_mean = 20
special_2_sd = 5

special_3_mean = 20
special_3_sd = 5

In [4]:
# The variables are:

# The number of doctors performing the initial assessments
num_doctors = 1
# The number of specialists treating the patients for each health condition
num_special_1 = 1
num_special_2 = 1
num_special_3 = 1

In [5]:
# Create the environment, the Clinic
class Clinic:
    def __init__(self, env, num_doctors, num_special_1, num_special_2, num_special_3):
        self.env = env
        self.doctor = simpy.Resource(env, num_doctors)
        self.special_1 = simpy.Resource(env, capacity=num_special_1)
        self.special_2 = simpy.Resource(env, capacity=num_special_2)
        self.special_3 = simpy.Resource(env, capacity=num_special_3)
    
    def initial_assess(self, patient):
        yield self.env.timeout(random.randint(assess_lower, assess_upper))
    
    def special_1(self, patient):
        yield self.env.timeout(random.normalvariate(special_1_mean, special_1_sd))
    
    def special_2(self, patient):
        yield self.env.timeout(random.normalvariate(special_2_mean, special_2_sd))
        
    def special_3(self, patient):
        yield self.env.timeout(random.normalvariate(special_3_mean, special_3_sd))

In [6]:
# Create a function to track a patient through the clinic
def visit_clinic(env, patient, clinic):
    # assign a condition to the patient
    condition_random = random.random()
    if condition_random <= cond_1_prob:
        patient_condition = 1
    elif condition_random <= cond_1_prob + cond_2_prob:
        patient_condition = 2
    else:
        patient_condition = 3
    
    # patient arrives at clinic
    assess_arrival_time = env.now
    
    with clinic.doctor.request() as request:
        yield request
        # time patient waited to be assessed
        assess_start_time = env.now
        assess_waiting = assess_start_time - assess_arrival_time
        
        yield env.process(clinic.initial_assess(patient))
    
    # patient completes assesment
    treat_arrival_time = env.now
    assessment_time = treat_arrival_time - assess_start_time
    
    # The patient is sent to the appropriate specialist
    if patient_condition == 1:
        with clinic.special_1.request() as request:
            yield request
            # time patient waited for treatment 1
            treat_start_time = env.now
            treat_waiting = treat_start_time - treat_arrival_time
            
            yield env.process(clinic.special_1(patient))
    
    elif patient_condition == 2:
        with clinic.special_2.request() as request:
            yield request
            # time patient waited for treatment 2
            treat_start_time = env.now
            treat_waiting = treat_start_time - treat_arrival_time
            
            yield env.process(clinic.special_2(patient))
    
    else:
        with clinic.special_3.request() as request:
            yield request
            # time patient waited for treatment 3
            treat_start_time = env.now
            treat_waiting = treat_start_time - treat_arrival_time
            
            yield env.process(clinic.special_3(patient))
    
    # patient completes treatment
    treat_comp_time = env.now
    treat_time = treat_comp_time - treat_start_time
    
    # save waiting times to list
    sim_waiting_times.append(assess_waiting+treat_waiting)
    
    # count number of patients treated
    patient_treat += 1

In [16]:
# Create a function to run the simulation
def run_clinic(env, num_doctors, num_special_1, num_special_2, num_special_3):
    clinic = Clinic(env, num_doctors, num_special_1, num_special_2, num_special_3)
    
    # allow for some patients to be waiting
    for patient in range(3):
        env.process(visit_clinic(env, patient, clinic))
    # simulate arrival of patients
    while True:
        yield env.timeout(random.expovariate(1/arrival_mean))
        
        patient += 1
        env.process(visit_clinic(env, patient, clinic))

In [17]:
# Run the simulation N times
def run_simulation():
    
    # run the simulation N times
    for i in range(N):
        
        # create simulation lists and counts
        sim_waiting_times = []
        patient_treat = 0
        
        # create environment
        env = simpy.Environment()
        env.process(run_clinic(env, num_doctors, num_special_1, num_special_2, num_special_3))
        env.run(until=420)
        
        # save performance indicators to lists
        total_patient.append(patient_treat)
        waiting_times.append(statistics.mean(sim_waiting_times))
        
    return total_patient, waiting_times
        
        


In [18]:
run_simulation()

TypeError: 'Resource' object is not callable